# BaseLine Modeling

## 1. setting modules

In [2]:
# load modules
import numpy as np
import pandas as pd

# split
from sklearn.model_selection import train_test_split

# models 
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# metrics
from sklearn.metrics import mean_squared_log_error

In [3]:
# load data
train = pd.read_csv('./data/external_open/countrywide_accident.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
# 필요한 데이터만 가져오기
use_train = train[['사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '시군구', '사망자수']]
use_test = test[test.columns.difference(['ID'])]

In [5]:
use_train

사고일시   요일 기상상태            도로형태   노면상태  사고유형             시군구  \
0       2019-01-01 00  화요일   맑음  교차로 - 교차로횡단보도내     건조  차대사람   서울특별시 강서구 방화동   
1       2019-01-01 00  화요일   맑음      교차로 - 교차로안     건조   차대차     경기도 포천시 소흘읍   
2       2019-01-01 00  화요일   맑음      교차로 - 교차로안     건조   차대차     경기도 양주시 고암동   
3       2019-01-01 00  화요일   맑음         기타 - 기타     건조  차대사람    경기도 남양주시 다산동   
4       2019-01-01 00  화요일   맑음      교차로 - 교차로안     건조   차대차   인천광역시 부평구 부평동   
...               ...  ...  ...             ...    ...   ...             ...   
602770  2021-12-31 23  금요일   흐림      교차로 - 교차로안     건조   차대차   부산광역시 해운대구 우동   
602771  2021-12-31 23  금요일   맑음        단일로 - 기타     건조   차대차   서울특별시 성동구 행당동   
602772  2021-12-31 23  금요일   맑음      교차로 - 교차로안     건조   차대차  대전광역시 유성구 원신흥동   
602773  2021-12-31 23  금요일   맑음         기타 - 기타  젖음/습기  차량단독     경기도 화성시 서신면   
602774  2021-12-31 23  금요일   맑음        단일로 - 기타     건조   차대차     경기도 파주시 와동동   

        사망자수  
0          0  
1          0  
2          0  
3          0  
4          0  
...      ...  
602770     0  
602771     0  
602772     0  
602773     0  
602774     0  

[602775 rows x 8 columns]

In [6]:
def wrangiling(data_set):
    print("==================")
    print("       Shape      ")
    print("==================")

    display(data_set.shape)

    print("==================")
    print("        Info      ")
    print("==================")

    display(data_set.info())

    print("==================")
    print("      Columns     ")
    print("==================")
    
    display(data_set.columns)

    print("==================")
    print("        NaN       ")
    print("==================")

    display(data_set.isna().sum())

    print("==================")
    print("     Duplicated   ")
    print("==================")
    
    display(data_set[data_set.duplicated()])

    print("==================")
    print("    Description   ")
    print("==================")

    display(data_set.describe())

    print("==================")
    print("      Unique      ")
    print("==================")

    display(data_set.nunique())

In [7]:
wrangiling(use_train)

       Shape      


(602775, 8)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602775 entries, 0 to 602774
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사고일시    602775 non-null  object
 1   요일      602775 non-null  object
 2   기상상태    602775 non-null  object
 3   도로형태    602775 non-null  object
 4   노면상태    602774 non-null  object
 5   사고유형    602775 non-null  object
 6   시군구     602775 non-null  object
 7   사망자수    602775 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 36.8+ MB


None

      Columns     


...

사고일시   요일 기상상태         도로형태 노면상태  사고유형               시군구  \
216     2019-01-01 16  화요일   맑음   교차로 - 교차로안   건조   차대차    부산광역시 부산진구 전포동   
222     2019-01-01 17  화요일   맑음     단일로 - 기타   건조   차대차      경기도 남양주시 진접읍   
629     2019-01-02 17  수요일   맑음  교차로 - 교차로부근   건조   차대차    인천광역시 미추홀구 주안동   
975     2019-01-03 11  목요일   맑음     단일로 - 기타   건조   차대차   서울특별시 성동구 성수동1가   
1190    2019-01-03 18  목요일   맑음     단일로 - 기타   건조   차대차  충청북도 청주시 서원구 분평동   
...               ...  ...  ...          ...  ...   ...               ...   
601942  2021-12-30 16  목요일   맑음     단일로 - 기타   건조   차대차       경기도 광명시 소하동   
601961  2021-12-30 17  목요일   맑음   교차로 - 교차로안   건조   차대차  경기도 고양시 일산동구 백석동   
602428  2021-12-31 14  금요일   맑음     단일로 - 기타   건조   차대차       경상남도 양산시 동면   
602522  2021-12-31 16  금요일   맑음     단일로 - 기타   건조   차대차        경기도 화성시 능동   
602617  2021-12-31 18  금요일   맑음   교차로 - 교차로안   건조  차대사람     부산광역시 금정구 부곡동   

        사망자수  
216        0  
222        0  
629        0  
975        0  
1190       0  
...      ...  
601942     0  
601961     0  
602428     0  
602522     0  
602617     0  

[1900 rows x 8 columns]

    Description   


사망자수
count  602775.000000
mean        0.014994
std         0.126911
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         7.000000

      Unique      


사고일시    26210
요일          7
기상상태        6
도로형태       12
노면상태        7
사고유형        4
시군구      4778
사망자수        8
dtype: int64

In [18]:
use_train[use_train["노면상태"].isna()]

사고일시   요일 기상상태      도로형태 노면상태  사고유형           시군구  사망자수
133800  2019-08-25 00  일요일    비  단일로 - 기타  NaN  차량단독  경상북도 경산시 진량읍     0

## 2. data preprocessing

In [19]:
use_train["도로형태"].str.

0         교차로 - 교차로횡단보도내
1             교차로 - 교차로안
2             교차로 - 교차로안
3                기타 - 기타
4             교차로 - 교차로안
               ...      
602770        교차로 - 교차로안
602771          단일로 - 기타
602772        교차로 - 교차로안
602773           기타 - 기타
602774          단일로 - 기타
Name: 도로형태, Length: 602775, dtype: object

In [171]:
# one hot encoding
onehot = pd.get_dummies(use_train[['기상상태', '도로형태', '노면상태', '사고유형']], drop_first=True)
use_train2 = pd.concat([use_train, onehot], axis = 1).drop(['기상상태', '도로형태', '노면상태', '사고유형'], axis = 1)

onehot_test = pd.get_dummies(test[['기상상태', '도로형태', '노면상태', '사고유형']], drop_first=True)
use_test2 = pd.concat([use_test, onehot], axis = 1).drop(['기상상태', '도로형태', '노면상태', '사고유형'], axis = 1)

In [172]:
# 요일 월~금: 0 토~일:1
use_train2['주말'] = np.where(use_train2['요일'].isin(['월요일', '화요일', '수요일', '목요일', '금요일']), 0, 1)
use_train3 = use_train2.drop(['요일'], axis = 1)

use_test2['주말'] = np.where(use_test2['요일'].isin(['월요일', '화요일', '수요일', '목요일', '금요일']), 0, 1)
use_test3 = use_test2.drop(['요일'], axis = 1)

In [173]:
# 사고일시 나누기

## date type data
use_train3['사고일시'] = pd.to_datetime(use_train3['사고일시'])
use_test3['사고일시'] = pd.to_datetime(use_test3['사고일시'])

In [174]:
# split year, month, day, hour
use_train3['year'] = use_train3['사고일시'].dt.year
use_train3['month'] = use_train3['사고일시'].dt.month
use_train3['day'] = use_train3['사고일시'].dt.day
use_train3['hour'] = use_train3['사고일시'].dt.hour

use_test3['year'] = use_test3['사고일시'].dt.year
use_test3['month'] = use_test3['사고일시'].dt.month
use_test3['day'] = use_test3['사고일시'].dt.day
use_test3['hour'] = use_test3['사고일시'].dt.hour

In [175]:
# 시군구 나누기
use_train3[['시', '구', '동가']] = use_train3['시군구'].str.split(' ', expand = True)
use_train4 = use_train3.drop(['사고일시', '시군구', '시', 'year'], axis = 1)

use_test3[['시', '구', '동가']] = use_test3['시군구'].str.split(' ', expand = True)
use_test4 = use_test3.drop(['사고일시', '시군구', '시', 'year'], axis = 1)

In [176]:
# type change
use_train4['구'] = use_train4['구'].astype('category')
use_train4['동가'] = use_train4['동가'].astype('category')

use_test4['구'] = use_test4['구'].astype('category')
use_test4['동가'] = use_test4['동가'].astype('category')

In [177]:
# 구와 동가 제거
use_train5 = use_train4[use_train4.columns.difference(['구', '동가'])]
use_test5 = use_test4[use_test4.columns.difference(['구', '동가'])]

## 3. Modeling

In [181]:
## 3. hold-out train test split
x_train, x_valid, y_train, y_valid = train_test_split(use_train4[use_train4.columns.difference(['사망자수'])],
                                                      use_train4['사망자수'],
                                                      test_size = 0.3,
                                                      random_state = 42
                                                    )

In [182]:
# category values
labels = x_train.dtypes.reset_index()
categorical_cols = list(labels[labels[0] == 'category'].index)

In [183]:
# model
xgb = XGBRegressor(objective='count:poisson',
                    random_state = 42,
                    use_label_encoder=False,
                    enable_categorical=True,
                    tree_method='hist'
                    )
lgbm = LGBMRegressor(
    objective='poisson',
    random_state = 42
)
cb = CatBoostRegressor(
    cat_features = categorical_cols,
    objective = 'Poisson',
    random_state = 42
)

In [203]:
# fitting
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train)
cb.fit(x_train, y_train,
        eval_set=(x_valid, y_valid),
        use_best_model = True,
        plot=True)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001259 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 27726, number of used features: 22
[LightGBM] [Info] Start training from score -4.887792


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8618622	test: 0.8619330	best: 0.8619330 (0)	total: 86.4ms	remaining: 1m 26s
1:	learn: 0.7484133	test: 0.7485016	best: 0.7485016 (1)	total: 137ms	remaining: 1m 8s
2:	learn: 0.6524432	test: 0.6526774	best: 0.6526774 (2)	total: 203ms	remaining: 1m 7s
3:	learn: 0.5660794	test: 0.5662360	best: 0.5662360 (3)	total: 267ms	remaining: 1m 6s
4:	learn: 0.4986906	test: 0.4989890	best: 0.4989890 (4)	total: 319ms	remaining: 1m 3s
5:	learn: 0.4414234	test: 0.4418494	best: 0.4418494 (5)	total: 373ms	remaining: 1m 1s
6:	learn: 0.3919861	test: 0.3925812	best: 0.3925812 (6)	total: 443ms	remaining: 1m 2s
7:	learn: 0.3500069	test: 0.3507135	best: 0.3507135 (7)	total: 470ms	remaining: 58.2s
8:	learn: 0.3134048	test: 0.3142272	best: 0.3142272 (8)	total: 530ms	remaining: 58.4s
9:	learn: 0.2815096	test: 0.2824084	best: 0.2824084 (9)	total: 605ms	remaining: 59.9s
10:	learn: 0.2537776	test: 0.2547556	best: 0.2547556 (10)	total: 674ms	remaining: 1m
11:	learn: 0.2304418	test: 0.2315149	best: 0.2315149 

In [209]:
# predict
pred_xgb = xgb.predict(x_valid)
pred_lgbm = lgbm.predict(x_valid)
pred_cb = cb.predict(x_valid)

In [210]:
# metrics
rmsle_xgb = mean_squared_log_error(y_valid, pred_xgb, squared = False)
rmsle_lgbm = mean_squared_log_error(y_valid, pred_lgbm, squared = False)
rmsle_cb = mean_squared_log_error(y_valid, pred_cb, squared = False)

print(f'xgboost : {rmsle_xgb}')
print(f'lightgbm : {rmsle_lgbm}')
print(f'catboost : {rmsle_cb}')

xgboost : 0.06535774685603425
lightgbm : 0.06426787080362707
catboost : 0.06283688668842609
